In [1]:
import warnings
warnings.filterwarnings(action='ignore') # To show code clearly

In [2]:
import os
import csv
import scipy
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import GEOparse
gse = GEOparse.get_GEO("GSE39445", annotate_gpl='GPL15331', silent=True)

### Pre-processing